## Libraries requirements

In [3]:
pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install fastparquet

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install mlxtend

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install polars

Note: you may need to restart the kernel to use updated packages.


In [7]:
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import fpgrowth
import polars as pl
import os
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np

In [8]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

c:\Users\cesar\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Load data

In [9]:
# Load the data using Polars, not necessary for Churn labelling
#os.chdir(r'"C:\Users\cesar\Dropbox\My PC (LAPTOP-GU3S2J8B)\Downloads\transaction_data.parquet"')
#data = pl.read_parquet(r"C:\Users\cesar\Dropbox\My PC (LAPTOP-GU3S2J8B)\Downloads\transaction_data.parquet")


#data_part = data

#data_part = data_part.lazy().with_columns(pl.col(['date_order', 'date_invoice']).str.strptime(pl.Date, "%Y-%m-%d")).collect()

In [10]:
#
df = pd.read_csv(r"C:\Users\cesar\Dropbox\My PC (LAPTOP-GU3S2J8B)\Downloads\transactions_dataset.csv",sep =';',dtype={
    'client_id': 'int',
    'product_id': 'int',
    'quantity': 'int',
    'branch_id': 'int',
})


## Churn rate

In [11]:
df

,date_order,date_invoice,product_id,client_id,sales_net,quantity,order_channel,branch_id
0,2017-09-25,2017-09-25,2376333,188502,155.44320,3,online,2732
1,2017-09-25,2017-09-25,2520527,835089,16.39440,3,at the store,10279
2,2017-09-25,2017-09-25,2536756,1977896,365.76624,21,online,6184
3,2017-09-25,2017-09-25,3028673,598816,8.59878,201,at the store,4200
4,2017-09-25,2017-09-25,203377,2086861,1093.37400,3,by phone,7051
...,...,...,...,...,...,...,...,...
63319310,2019-06-10,2020-06-12,1596284,245272,-2056.91300,5,by phone,7682
63319311,2018-12-14,2020-07-15,908253,880496,27.62760,1001,by phone,9375
63319312,2019-07-26,2020-10-29,1168247,2063314,-204.60984,101,by phone,3014
63319313,2018-05-23,2020-11-27,2319965,1287654,70.49730,17,by phone,1015


In [12]:
# Method 1 Use pandas directly
# Problem with the types of int columns
#df
# Randomly select 100 000 element of my dataframe
df_sub = df.sample(n=100000)

In [66]:
df_sub

,date_order,date_invoice,product_id,client_id,sales_net,quantity,order_channel,branch_id
53515409,2019-06-14,2019-06-14,2906774,1808238,385.188360,201,online,9955
4465467,2017-11-14,2017-11-14,2676931,1177816,160.135200,3,at the store,8807
44074103,2019-02-20,2019-02-20,1109373,1399590,29.375600,103,online,7678
45561121,2019-03-06,2019-03-06,2620991,537705,16.394400,11,at the store,7203
40348903,2018-12-31,2018-12-31,669067,1228448,6.596400,3,at the store,4842
...,...,...,...,...,...,...,...,...
56102611,2019-07-11,2019-07-11,897211,1160407,59.036400,3,by phone,10964
55653237,2019-06-28,2019-07-04,1325810,1821278,78.831396,201,by phone,832
54495581,2019-06-24,2019-06-24,2405839,50667,26.937600,3,at the store,9818
11406097,2018-01-31,2018-02-02,2053373,1693884,40.737600,3,at the store,8539


In [109]:
def weekly_data(df):
    #df['date_order'] = pd.to_datetime(df['date_order'])
    #df = df.set_index("date_order")

# Resample the data per week on client_id and sum the quantity and net_sales columns
    weekly_data = df.groupby(["client_id", pd.Grouper(key="date_order",freq='W')]).agg({
        "quantity": "sum",
        "sales_net": "sum",
    })
    #df_1 = df.groupby(['client_id', pd.Grouper(key='date_order', freq='W-MON')]).agg(
    #quantity = ('quantity','sum'),
    #sales_net = ('sales_net','sum')).reset_index().sort_values('date_order')
    #print(weekly_data)
    # Get the counts per week for each order channel
    sales_channels = df.groupby(["client_id", pd.Grouper(freq="W-Mon", key="date_order")])["order_channel"].value_counts().unstack(fill_value=0)
    #print(sales_channels)
    # Merge the above dataframes together
    df = weekly_data.merge(sales_channels, left_on=["client_id", "date_order"], right_on=["client_id", "date_order"], how="left")
    df = pd.DataFrame(df)
    return df

In [111]:
df_1 = weekly_data(df_sub)

df_1

quantity    sales_net  at the store  by phone  \
client_id date_order                                                  
14        2017-10-01         3    28.869600           NaN       NaN   
          2017-12-10         3  1767.669600           NaN       NaN   
          2017-12-17         3    34.803600           NaN       NaN   
          2018-02-18      1001    98.335838           NaN       NaN   
          2018-03-11         7    33.874400           NaN       NaN   
...                        ...          ...           ...       ...   
2274299   2018-07-22         3    21.500400           NaN       NaN   
          2018-08-26         3    20.920800           NaN       NaN   
          2018-11-25        31   149.045520           NaN       NaN   
          2019-06-16         3    54.040800           NaN       NaN   
2274322   2019-03-31        31     0.000000           NaN       NaN   

                      during the visit of a sales rep  online  other  
client_id date_order                                                  
14        2017-10-01                              NaN     NaN    NaN  
          2017-12-10                              NaN     NaN    NaN  
          2017-12-17                              NaN     NaN    NaN  
          2018-02-18                              NaN     NaN    NaN  
          2018-03-11                              NaN     NaN    NaN  
...                                               ...     ...    ...  
2274299   2018-07-22                              NaN     NaN    NaN  
          2018-08-26                              NaN     NaN    NaN  
          2018-11-25                              NaN     NaN    NaN  
          2019-06-16                              NaN     NaN    NaN  
2274322   2019-03-31                              NaN     NaN    NaN  

[94580 rows x 7 columns]

In [80]:
sub_1 = df_sub.groupby(["client_id", pd.Grouper(freq="W", key="date_order")]).agg({
        "quantity": "sum",
        "sales_net": "sum",
    })
sub_1

quantity    sales_net
client_id date_order                       
14        2017-10-01         3    28.869600
          2017-12-10         3  1767.669600
          2017-12-17         3    34.803600
          2018-02-18      1001    98.335838
          2018-03-11         7    33.874400
...                        ...          ...
2274299   2018-07-22         3    21.500400
          2018-08-26         3    20.920800
          2018-11-25        31   149.045520
          2019-06-16         3    54.040800
2274322   2019-03-31        31     0.000000

[94580 rows x 2 columns]

In [58]:

#df_sub_1 = df_sub.groupby(['client_id', pd.Grouper(key='date_order', freq='W-MON')]).agg(
    #'quanity = ['quantity'].sum().reset_index().sort_values('date_order')
df_sub_1 = df_sub.groupby(['client_id', pd.Grouper(key='date_order', freq='W-MON')]).agg(
    quantity = ('quantity','sum'),
    sales_net = ('sales_net','sum')).reset_index().sort_values('date_order')
df_sub_1

,client_id,date_order,quantity,sales_net
4039,97857,2017-09-25,3,2.29080
22019,521451,2017-09-25,21,83.32716
88471,2121520,2017-09-25,3,157.40280
4085,99019,2017-09-25,11,1.51800
56481,1364727,2017-09-25,3,140.12520
...,...,...,...,...
77096,1840345,2019-09-23,5,8.57900
22487,533100,2019-09-23,3,57.02160
70231,1682126,2019-09-23,3,316.07520
52684,1274642,2019-09-23,602,79.67568


In [ ]:
def clean_data(df):
    df.drop('product_id', axis =1)
    return df

In [ ]:
def order_history(df):
    df['date_order'] = pd.to_datetime(df['date_order'])
    df['week'] = df['date_order'].dt.isocalendar().week
    df = df.sort_values(['client_id', 'date_order'])
    
    # create a new dataframe to hold the time deltas
    time_diffs = df.groupby('client_id')['date_order'].diff().fillna(pd.Timedelta(0))
    # add a new column to the dataframe with the number of weeks since each client's last transaction
    df['weeks_since_last_transaction'] = df['time_diff'].dt.days /7

    # fill any missing values with 0
    df['weeks_since_last_transaction'] = df['weeks_since_last_transaction'].fillna(0)
    df['avg_time_between_transactions'] = df.groupby('client_id')['weeks_since_last_transaction'].transform('mean')
    df['max_time_between_transactions'] = df.groupby('client_id')['weeks_since_last_transaction'].transform('max')
    return df

In [ ]:
def churn(df):
    df = order_history(df)
    df['churn']= np.where(df['weeks_since_last_transaction']>
                               3*df['avg_time_between_transactions'] or 1,5 *df['max_time_between_transactions'] ,1,0)
    return df

In [112]:
def find_churn_dates(df):
    # Sort the data by client_id and date
    df = df.sort_values(['client_id', 'date'])

    # Create a mask for the rows where churn is 0
    mask_0 = df['churn'] == 0

    # Create a mask for the rows where churn is 1
    mask_1 = df['churn'] == 1

    # Get the rows with the latest date where churn is 0 for each client_id
    latest_0 = df[mask_0].drop_duplicates(subset='client_id', keep='last').drop(columns=['churn'])

    # Get the rows with the latest date where churn is 1 for each client_id
    latest_1 = df[mask_1].drop_duplicates(subset='client_id', keep='last').drop(columns=['churn'])

    # Combine the rows where churn is 0 and churn is 1
    result = pd.concat([latest_0, latest_1]).sort_index()
    result.drop('date',axis=1)
    return result

In [113]:
find_churn_dates(df_sub)

KeyError: 'date'

In [ ]:
def add_additional_data(df):
    df['total_quantity']=df.groupby('client_id')['quantity'].transform('sum')
    df['avg_quantity']=df.groupby('client_id')['quantity'].transform('mean')
    df['total_sales_net']=df.groupby('client_id')['sales_net'].transform('sum')
    df['quantity_by_week'] = df.groupby(['client_id', pd.Grouper(key='date_order', freq='W-MON')])['quantity'].transform('sum')
df_sub

,date_order,date_invoice,product_id,client_id,sales_net,quantity,order_channel,branch_id,week,time_diff,weeks_since_last_transaction,avg_time_between_transactions,Churn,churn,date,total_quantity,total_sales_net,avg_quantity,quantity_by_week
16460798,2018-03-30,2018-03-30,284038,14,67.235900,41,by phone,10319,13,0 days,0.000000,0.000000,0,0,2018-03-30,41,67.235900,41.0,41
11901845,2018-02-08,2018-02-08,2322150,485,81.223120,11,by phone,9526,6,0 days,0.000000,36.357143,0,0,2018-02-08,14,98.473120,7.0,11
55289568,2019-07-02,2019-07-02,303740,485,17.250000,3,at the store,9526,27,509 days,72.714286,36.357143,0,0,2019-07-02,14,98.473120,7.0,3
40949299,2019-01-14,2019-01-14,877703,843,1170.074400,3,by phone,7101,3,0 days,0.000000,0.000000,0,0,2019-01-14,3,1170.074400,3.0,3
17885888,2018-04-06,2018-04-12,2827358,998,159.965000,13,at the store,4157,14,0 days,0.000000,0.000000,0,0,2018-04-06,13,159.965000,13.0,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33574513,2018-10-18,2018-10-18,1387576,2273952,158.893200,3,at the store,9904,42,0 days,0.000000,0.000000,0,0,2018-10-18,3,158.893200,3.0,3
35558378,2018-11-12,2018-11-12,348995,2274084,8.932924,1001,by phone,2255,46,0 days,0.000000,0.000000,0,0,2018-11-12,1001,8.932924,1001.0,1001
23326928,2018-06-27,2018-06-27,359388,2274156,12.613200,3,online,4424,26,0 days,0.000000,0.000000,0,0,2018-06-27,3,12.613200,3.0,3
25948135,2018-07-26,2018-07-26,1202537,2274190,108.249960,21,by phone,5122,30,0 days,0.000000,0.000000,0,0,2018-07-26,21,108.249960,21.0,21


In [ ]:
find_churn_dates(df_sub)

,date_order,date_invoice,product_id,client_id,sales_net,quantity,order_channel,branch_id,week,time_diff,weeks_since_last_transaction,avg_time_between_transactions,Churn,date
1188,2017-09-25,2017-09-25,3000784,1553299,2.290800,3,at the store,5675,39,0 days,0.000000,0.000000,0,2017-09-25
2910,2017-09-29,2017-09-29,177199,1611171,19.590480,21,at the store,977,39,0 days,0.000000,0.000000,0,2017-09-29
9738,2017-09-26,2017-09-26,1723510,1066533,15.469289,37,at the store,5395,39,0 days,0.000000,0.000000,0,2017-09-26
35248,2017-10-02,2017-10-02,1350410,649578,21.451824,51,at the store,10236,40,0 days,0.000000,0.000000,0,2017-10-02
36078,2017-09-27,2017-09-27,519804,2173561,23.009200,41,by phone,9668,39,0 days,0.000000,0.000000,0,2017-09-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63307181,2019-08-13,2020-01-20,1575356,972334,86.897496,301,by phone,358,33,176 days,25.142857,12.571429,0,2019-08-13
63308969,2019-07-30,2020-03-05,661878,941849,118.105000,5,at the store,8828,31,656 days,93.714286,46.857143,0,2019-07-30
63311324,2019-09-12,2020-06-03,334090,2029984,-336.720000,3,by phone,1894,37,0 days,0.000000,0.000000,0,2019-09-12
63318483,2019-08-30,2020-02-18,2958632,1386080,592.020000,39,by phone,3175,35,0 days,0.000000,0.000000,0,2019-08-30


In [ ]:
def find_latest_rows(df):
    df = churn(df)
    churn_0_latest = df[df['churn'] == 0].sort_values('date_order', ascending=False).iloc[0]
    churn_1_latest = df[df['churn'] == 1].sort_values('date_order', ascending=False).iloc[0]
    return pd.concat([churn_0_latest, churn_1_latest], axis=1)


In [ ]:
def order_invoice_diff(df):
    df['order_invoice_diff'] = (pd.to_datetime(df['order_date']) - pd.to_datetime(df['invoice_date'])).dt.days
    return df

In [ ]:
def get_churn_rows(df):
    # Sort the DataFrame by date and churn
    df = df.sort_values(['client_id', 'date_order', 'churn'])

    # Group by client_id and get the first and last rows for each group
    churn_rows = df.groupby('client_id').agg(first_row=('date_order', 'first'),
                                             last_row=('date_order', 'last'),
                                             first_churn=('churn', 'first'),
                                             last_churn=('churn', 'last'))

    # Return the rows where churn is 0 and 1
    churn_0 = churn_rows[churn_rows['last_churn'] == 0]['last_row']
    churn_1 = churn_rows[churn_rows['first_churn'] == 1]['first_row']

    # Combine the results and return them
    result = pd.concat([churn_0, churn_1], axis=0)
    return result

In [ ]:

def get_churn_dates(df):
    df['date'] = pd.to_datetime(df['date_order']) # convert date column to datetime type
    df = df.sort_values(by=['client_id', 'date_order'], ascending=[True, False]) # sort dataframe by client_id and date in descending order

    churn_0 = df[df['churn'] == 0].groupby('client_id').first() # get first row of each client_id where churn = 0
    churn_1 = df[df['churn'] == 1].groupby('client_id').first() # get first row of each client_id where churn = 1

    merged = pd.merge(churn_0, churn_1, on='client_id', how='outer', suffixes=('_churn0', '_churn1')) # merge the two dataframes on client_id

    # fill missing values with the last non-null value
    merged = merged.ffill()

    # drop unnecessary columns and rename remaining ones
    merged = merged.drop(['date_churn0', 'churn_churn0', 'date_churn1', 'churn_churn1'], axis=1)
    merged = merged.rename(columns={'date_churn0_ffill': 'last_transaction_date_churn0', 'date_churn1_ffill': 'last_transaction_date_churn1', 'churn_churn1_ffill': 'churn'})

    return merged


In [ ]:
get_churn_dates(df_sub)

,date_order_churn0,date_invoice_churn0,product_id_churn0,sales_net_churn0,quantity_churn0,order_channel_churn0,branch_id_churn0,week_churn0,time_diff_churn0,weeks_since_last_transaction_churn0,...,product_id_churn1,sales_net_churn1,quantity_churn1,order_channel_churn1,branch_id_churn1,week_churn1,time_diff_churn1,weeks_since_last_transaction_churn1,avg_time_between_transactions_churn1,Churn_churn1
client_id,,,,,,,,,,,,,,,,,,,,,
14,2018-03-30,2018-03-30,284038,67.235900,41,by phone,10319,13,0 days,0.000000,...,NaN,NaN,NaN,NaN,NaN,<NA>,NaT,NaN,NaN,NaN
485,2019-07-02,2019-07-02,303740,17.250000,3,at the store,9526,27,509 days,72.714286,...,NaN,NaN,NaN,NaN,NaN,<NA>,NaT,NaN,NaN,NaN
843,2019-01-14,2019-01-14,877703,1170.074400,3,by phone,7101,3,0 days,0.000000,...,NaN,NaN,NaN,NaN,NaN,<NA>,NaT,NaN,NaN,NaN
998,2018-04-06,2018-04-12,2827358,159.965000,13,at the store,4157,14,0 days,0.000000,...,NaN,NaN,NaN,NaN,NaN,<NA>,NaT,NaN,NaN,NaN
1066,2019-03-18,2019-03-18,2894395,7.785132,201,at the store,4747,12,0 days,0.000000,...,NaN,NaN,NaN,NaN,NaN,<NA>,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273952,2018-10-18,2018-10-18,1387576,158.893200,3,at the store,9904,42,0 days,0.000000,...,1727052.0,433.251,65.0,online,1888.0,28,242 days,34.571429,10.836735,1.0
2274084,2018-11-12,2018-11-12,348995,8.932924,1001,by phone,2255,46,0 days,0.000000,...,1727052.0,433.251,65.0,online,1888.0,28,242 days,34.571429,10.836735,1.0
2274156,2018-06-27,2018-06-27,359388,12.613200,3,online,4424,26,0 days,0.000000,...,1727052.0,433.251,65.0,online,1888.0,28,242 days,34.571429,10.836735,1.0


In [ ]:
get_churn_rows(df_sub)

client_id
14        2018-03-30
485       2019-07-02
843       2019-01-14
998       2018-04-06
1066      2019-03-18
             ...    
2273952   2018-10-18
2274084   2018-11-12
2274156   2018-06-27
2274190   2018-07-26
2274299   2019-02-04
Length: 7498, dtype: datetime64[ns]

In [ ]:
# group the DataFrame by 'client_id' and get the index of the latest transaction for each group
#latest_idx = df.groupby('client_id')['date_order'].max()
df['is_latest'] = np.where(df.index == df.groupby('client_id')['date_order'].max(),1,0)
# create a new column with a boolean value that is True for the latest transaction and False for all others
#df['is_latest'] =np.where(df.index == latest_idx,1,0)

ValueError: Lengths must match to compare

In [ ]:
order_history(df_sub)


,date_order,date_invoice,product_id,client_id,sales_net,quantity,order_channel,branch_id,week,time_diff,weeks_since_last_transaction,avg_time_between_transactions,Churn
16460798,2018-03-30,2018-03-30,284038,14,67.235900,41,by phone,10319,13,0 days,0.000000,0.000000,0
11901845,2018-02-08,2018-02-08,2322150,485,81.223120,11,by phone,9526,6,0 days,0.000000,36.357143,0
55289568,2019-07-02,2019-07-02,303740,485,17.250000,3,at the store,9526,27,509 days,72.714286,36.357143,0
40949299,2019-01-14,2019-01-14,877703,843,1170.074400,3,by phone,7101,3,0 days,0.000000,0.000000,0
17885888,2018-04-06,2018-04-12,2827358,998,159.965000,13,at the store,4157,14,0 days,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33574513,2018-10-18,2018-10-18,1387576,2273952,158.893200,3,at the store,9904,42,0 days,0.000000,0.000000,0
35558378,2018-11-12,2018-11-12,348995,2274084,8.932924,1001,by phone,2255,46,0 days,0.000000,0.000000,0
23326928,2018-06-27,2018-06-27,359388,2274156,12.613200,3,online,4424,26,0 days,0.000000,0.000000,0
25948135,2018-07-26,2018-07-26,1202537,2274190,108.249960,21,by phone,5122,30,0 days,0.000000,0.000000,0


In [ ]:
def churn_rate(df):
    rate = df['Churn_rate'].sum()/df['Churn_rate'].count()
    return rate

In [ ]:
import numpy as np
df_sub['Churn']= np.where(df_sub['weeks_since_last_transaction']> 3*df_sub['avg_time_between_transactions'],1,0)

In [ ]:
df[-1:]['date_order']

63319314    2019-09-10
Name: date_order, dtype: object

In [ ]:
def end_date(df):
    end = df[-1:]['date_order']
    return end

In [ ]:
df_sub['Churn'].count()

10000

In [ ]:
a['date_order']=pl.date(a['date_order'])
a['week'] = a['date_order'].dt.week()

In [ ]:
def weekly_grouping(df):
    df.lazy.groupby(['date_order','client_id'])
    df = df.lazy().groupby_dynamic('date_order',every = '7d').agg([pl.col('quantity').sum().alias('sum_quantity'),
                                                 pl.col('quantity').mean().alias('mean_quantity'),
                                                 pl.col('sales_net').sum().alias('sum_sales_net'),
                                                 pl.col('sales_net').mean().alias('mean_sales_net')]
                                                 ).collect()
    return df


In [ ]:
weekly_grouping(data_part)

In [ ]:
#new_df = data_part.lazy().groupby('client_id').agg('quantity').sort("sum")
new_df = data_part.groupby('client_id').count()#['quantity']
new_df

In [ ]:
counts = data_part.lazy().groupby('client_id').sort('client_id')

# sort the counts in descending order
#counts = counts.sort()
# create a new dataframe to hold the time differences
time_diffs = pl.DataFrame(columns=['customer_id', 'time_diff'])

# loop through each customer's transaction history
for customer_id in counts.index:
    customer_df = data_part.filter(pl.col('customer_id') == customer_id).sort('date_order')
    time_diff = customer_df['date'].diff().shift(-1)
    time_diffs = time_diffs.append(pl.DataFrame({'customer_id': customer_id, 'time_diff': time_diff}))

# merge the time differences with the original dataframe
df = df.merge(time_diffs, on=['customer_id', 'date'], how='left')

# fill any missing values with 0 (i.e. there was no previous transaction for that customer)
df['time_diff'] = df['time_diff'].fillna(pl.Timedelta(0))

# add a new column to the dataframe with the number of days since each customer's last transaction
df['days_since_last_transaction'] = df['time_diff'].apply(lambda x: x.days)

In [ ]:
df_1 = data_part.lazy().groupby('client_id').agg([pl.col('quantity').sum().alias('sum_quantity'),
                                                 pl.col('quantity').mean().alias('mean_quantity'),
                                                 pl.col('sales_net').sum().alias('sum_sales_net'),
                                                 pl.col('sales_net').mean().alias('mean_sales_net')]
                                                 ).collect()
df_1

In [ ]:
def detect_churn(df):
    now = df['date_order'].max()

    three_months_ago = now - timedelta(days=90)

    latest_order_date = df.lazy().groupby('client_id').agg([pl.max('date_order')]).collect()

    q = latest_order_date.select(
            [
                pl.col("*"),
                pl.when(pl.col('date_order') > three_months_ago).then(0).otherwise(1)
        ]
    )

    return q

In [ ]:

churned_weekly['total_num_churn_week'] = churned.sort('date_order').groupby_dynamic("date_order", every="7d").agg(pl.col("literal").count())
quantity_weekly['total_quantity_week'] = data_part.sort('date_order').groupby_dynamic('date_order', every='7d').agg(pl.col('quantity').mean())
churned_weekly

In [ ]:
sales_channel = data_part.lazy().groupby(['date_order', 'order_channel']).agg(pl.col('sales_net').sum()).collect()
quantity_channel = data_part.lazy().groupby(['date_order', 'order_channel']).agg(pl.col('quantity').sum()).collect()

In [ ]:
sales_channel_weekly = sales_channel.sort('date_order').groupby_dynamic('date_order', every='7d').agg(
    [
        pl.col('sales_net').filter(pl.col('order_channel') == 'online').mean().alias('online'),
        pl.col('sales_net').filter(pl.col('order_channel') == 'by phone').mean().alias('by phone'),
        pl.col('sales_net').filter(pl.col('order_channel') == 'at the store').mean().alias('at the store'),
        pl.col('sales_net').filter(pl.col('order_channel') == 'during the visit of a sales rep').mean().alias('during the visit of a sales rep')
    ]
    ).sort(('date_order'))

In [ ]:
#sales_channel_weekly

In [ ]:
churned = detect_churn(data_part)

churned.lazy().groupby('date_order').agg(
    [
        pl.count('literal')
    ]
).sort('date_order')

churned_weekly = churned.sort('date_order').groupby_dynamic("date_order", every="7d").agg(pl.col("literal").count())
quantity_weekly = data_part.sort('date_order').groupby_dynamic('date_order', every='7d').agg(pl.col('quantity').mean())
sales_channel = data_part.lazy().groupby(['date_order', 'order_channel']).agg(pl.col('sales_net').sum()).collect()
quantity_channel = data_part.lazy().groupby(['date_order', 'order_channel']).agg(pl.col('quantity').sum()).collect()

plt.plot(churned_weekly['date_order'], churned_weekly['literal'])
plt.plot(quantity_weekly['date_order'], quantity_weekly['quantity'])


sales_channel_weekly = sales_channel.sort('date_order').groupby_dynamic('date_order', every='7d').agg(
    [
        pl.col('sales_net').filter(pl.col('order_channel') == 'online').mean().alias('online'),
        pl.col('sales_net').filter(pl.col('order_channel') == 'by phone').mean().alias('by phone'),
        pl.col('sales_net').filter(pl.col('order_channel') == 'at the store').mean().alias('at the store'),
        pl.col('sales_net').filter(pl.col('order_channel') == 'during the visit of a sales rep').mean().alias('during the visit of a sales rep')
    ]
    ).sort(('date_order'))

quantity_channel_weekly = quantity_channel.sort('date_order').groupby_dynamic('date_order', every='7d').agg(
    [
        pl.col('quantity').filter(pl.col('order_channel') == 'online').mean().alias('online'),
        pl.col('quantity').filter(pl.col('order_channel') == 'by phone').mean().alias('by phone'),
        pl.col('quantity').filter(pl.col('order_channel') == 'at the store').mean().alias('at the store'),
        pl.col('quantity').filter(pl.col('order_channel') == 'during the visit of a sales rep').mean().alias('during the visit of a sales rep')
    ]
    ).sort(('date_order'))



In [ ]:
new_df = data_part.groupby('client_id').agg('quantity')

In [ ]:
time_diffs = pl.DataFrame(columns=['customer_id', 'time_diff'])

# loop through each customer's transaction history
for customer_id in counts.index:
    customer_df = df[df['customer_id'] == customer_id].sort_values('date')
    time_diff = customer_df['date'].diff().shift(-1)
    time_diffs = time_diffs.append(pl.DataFrame({'customer_id': customer_id, 'time_diff': time_diff}))

In [ ]:
def detect_churn(df):
    now = df['date_order'].max()

    three_months_ago = now - timedelta(days=90)

    latest_order_date = df.lazy().groupby('client_id').agg([pl.max('date_order')]).collect()

    q = latest_order_date.select(
            [
                pl.col("*"),
                pl.when(pl.col('date_order') > three_months_ago).then(0).otherwise(1)
        ]
    )
    return q

In [ ]:


def bar_char_max_churn(df):
# Assuming the Dataframe is called df
    agg_df = df.groupby('date_order').sum()
    churn_count = agg_df[agg_df['literal'] == 0].sort_values(by='date_order', ascending=False)

    plt.bar(churn_count['date'], churn_count['literal'])
    plt.title("Churn rate analysis")
    plt.xlabel("Date")
    plt.ylabel("Number of churns")

    plt.show()


In [ ]:
#





churned = detect_churn(data_part)

churned.lazy().groupby('date_order').agg(
    [
        pl.count('literal')
    ]
).sort('date_order')

churned_weekly = churned.sort('date_order').groupby_dynamic("date_order", every="7d").agg(pl.col("literal").count())
quantity_weekly = data_part.sort('date_order').groupby_dynamic('date_order', every='7d').agg(pl.col('quantity').mean())
sales_channel = data_part.lazy().groupby(['date_order', 'order_channel']).agg(pl.col('sales_net').sum()).collect()
quantity_channel = data_part.lazy().groupby(['date_order', 'order_channel']).agg(pl.col('quantity').sum()).collect()

plt.plot(churned_weekly['date_order'], churned_weekly['literal'])
plt.plot(quantity_weekly['date_order'], quantity_weekly['quantity'])


sales_channel_weekly = sales_channel.sort('date_order').groupby_dynamic('date_order', every='7d').agg(
    [
        pl.col('sales_net').filter(pl.col('order_channel') == 'online').mean().alias('online'),
        pl.col('sales_net').filter(pl.col('order_channel') == 'by phone').mean().alias('by phone'),
        pl.col('sales_net').filter(pl.col('order_channel') == 'at the store').mean().alias('at the store'),
        pl.col('sales_net').filter(pl.col('order_channel') == 'during the visit of a sales rep').mean().alias('during the visit of a sales rep')
    ]
    ).sort(('date_order'))

quantity_channel_weekly = quantity_channel.sort('date_order').groupby_dynamic('date_order', every='7d').agg(
    [
        pl.col('quantity').filter(pl.col('order_channel') == 'online').mean().alias('online'),
        pl.col('quantity').filter(pl.col('order_channel') == 'by phone').mean().alias('by phone'),
        pl.col('quantity').filter(pl.col('order_channel') == 'at the store').mean().alias('at the store'),
        pl.col('quantity').filter(pl.col('order_channel') == 'during the visit of a sales rep').mean().alias('during the visit of a sales rep')
    ]
    ).sort(('date_order'))


def plot_stackgraph(df: pl.DataFrame,
          title: str) -> type[plt.figure]:
    _, ax = plt.subplots()
    for cols in df.columns[1:]:
        ax.plot(df['date_order'], df[cols], label=cols)
    
    ax.legend()
    plt.xlabel('Date')
    plt.xticks(rotation=90)
    plt.ylabel('Quantity')
    plt.title(title)
    plt.show()

plot_stackgraph(quantity_channel_weekly,
                title='Average weekly quantity sold per sales channel')
plot_stackgraph(sales_channel_weekly,
                title='Average weekly net sales per channel')


### FP-Growth analysis ###
df  = data_part[0:10_000][["order_channel", "client_id", "quantity"]]
df = pd.DataFrame(df).T
df = df.rename(columns={0: "order_channel",
                        1: "client_id",
                        2: "quantity"})

basket = (df.groupby(["order_channel", "client_id"])["quantity"]
            .sum().unstack().reset_index().fillna(0)
            .set_index("order_channel"))

basket = basket.astype("bool")

frequent_item_sets = fpgrowth(basket, min_support=0.05, use_colnames=True)
frequent_item_sets.sort_values(by="support", ascending=False).head(10)

df.client_id.value_counts()